In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

CATALOG_NAME = "local"
DB_NAME = "test_db"
TABLE_NAME = "customer_nopartitions"

def initialize_spark():
    # Initialize Spark session with Iceberg catalog configuration
    return SparkSession.builder \
        .appName("IcebergTableCreation") \
        .master("local[*]") \
        .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config(f"spark.sql.catalog.{CATALOG_NAME}", "org.apache.iceberg.spark.SparkCatalog") \
        .config(f"spark.sql.catalog.{CATALOG_NAME}.type", "hadoop") \
        .config(f"spark.sql.catalog.{CATALOG_NAME}.warehouse", "file:///path/to/warehouse") \
        .getOrCreate()

def define_schema():
    # Define the schema for your DataFrame
    schema = StructType([
        StructField("c_customer_sk", IntegerType(), False),
        StructField("c_customer_id", StringType(), False),
        StructField("c_first_name", StringType(), True),
        StructField("c_last_name", StringType(), True),
        StructField("c_birth_country", StringType(), True),
        StructField("c_email_address", StringType(), True)
    ])
    return schema

def create_dataframe(spark, schema):
    # Create some sample data
    data = [
        (1, "ID_001", "John", "Doe", "USA", "john.doe@example.com"),
        (2, "ID_002", "Jane", "Smith", "Canada", "jane.smith@example.com")
    ]
    
    # Create DataFrame with the schema
    df = spark.createDataFrame(data, schema)
    return df

def create_iceberg_table(spark):
    # Create Iceberg table using SQL
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DB_NAME}.{TABLE_NAME} (
            c_customer_sk             int,
            c_customer_id             string,
            c_first_name              string,
            c_last_name               string,
            c_birth_country           string,
            c_email_address           string)
        USING iceberg
        OPTIONS ('format-version'='2')
    """)

def insert_data_into_iceberg_table(df):
    # Insert data into Iceberg table
    df.writeTo(f"{CATALOG_NAME}.{DB_NAME}.{TABLE_NAME}").append()

def main():
    spark = initialize_spark()
    schema = define_schema()
    df = create_dataframe(spark, schema)
    
    create_iceberg_table(spark)
    insert_data_into_iceberg_table(df)
    
    print("Data successfully written to the Iceberg table.")
    
    # Verify the data
    spark.sql(f"SELECT * FROM {CATALOG_NAME}.{DB_NAME}.{TABLE_NAME}").show()

if __name__ == "__main__":
    main()

25/01/09 17:03:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Py4JJavaError: An error occurred while calling o30.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'local': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1926)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:54)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:54)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndIdentifier$.unapply(LookupCatalog.scala:122)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:36)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:30)
	at or

In [8]:
# Assume df is your existing PySpark DataFrame
df = spark.createDataFrame([
    (1, "Alice", 29),
    (2, "Bob", 30),
    (3, "Cathy", 25)
], ["id", "name", "age"])

# Write DataFrame to Iceberg table
df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .save("spark_catalog.default.my_iceberg_table")

Py4JJavaError: An error occurred while calling o132.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: iceberg. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:260)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:243)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.in

In [3]:
iceberg_df = spark.read \
    .format("iceberg") \
    .load("spark_catalog.default.my_iceberg_table")

iceberg_df.show()

Py4JJavaError: An error occurred while calling o55.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: iceberg. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at j

In [6]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg catalog configuration
spark = SparkSession.builder \
    .appName("IcebergExample") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "file:///path/to/warehouse") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark3-runtime:0.12.1") \
    .getOrCreate()

25/01/09 11:29:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
